In [19]:
##  Some imports and making connection ..
from pymilvus import (connections, utility, FieldSchema, CollectionSchema, DataType, Collection)
connections.connect("default", host="localhost", port="19530")

In [20]:
# Createing a collection ..
fields = [FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
          FieldSchema(name="random", dtype=DataType.DOUBLE),
          FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=8)]

fields

[{
     name: pk
     description: 
     type: 5
     is_primary: True
     auto_id: False
   },
 {
     name: random
     description: 
     type: 11
   },
 {
     name: embeddings
     description: 
     type: 101
     params: {'dim': 8}
   }]

In [21]:
schema = CollectionSchema(fields, "hello_milvus is the simplest demo to introduce the APIs")
schema

{
  auto_id: False
  description: hello_milvus is the simplest demo to introduce the APIs
  fields: [{
    name: pk
    description: 
    type: 5
    is_primary: True
    auto_id: False
  }, {
    name: random
    description: 
    type: 11
  }, {
    name: embeddings
    description: 
    type: 101
    params: {'dim': 8}
  }]
}

In [22]:
hello_milvus = Collection("hello_milvus", schema)
hello_milvus

<Collection>:
-------------
<name>: hello_milvus
<partitions>: [{"name": "_default", "collection_name": "hello_milvus", "description": ""}]
<description>: hello_milvus is the simplest demo to introduce the APIs
<schema>: {
  auto_id: False
  description: hello_milvus is the simplest demo to introduce the APIs
  fields: [{
    name: pk
    description: 
    type: 5
    is_primary: True
    auto_id: False
  }, {
    name: random
    description: 
    type: 11
  }, {
    name: embeddings
    description: 
    type: 101
    params: {'dim': 8}
  }]
}

In [23]:
# Creating arrays ..
import random
entities = [[i for i in range(3000)],  # field pk
            [float(random.randrange(-20, -10)) for _ in range(3000)],  # field random
            [[random.random() for _ in range(8)] for _ in range(3000)],  # field embeddings
           ]

# inserting all vectors to collection
insert_result = hello_milvus.insert(entities)
hello_milvus.flush()  

In [24]:
# Builds indexes on the entities ..
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},}

hello_milvus.create_index("embeddings", index)

Status(code=0, message=)

In [25]:
# Loads the collection to memory and performs a vector similarity search ..
hello_milvus.load()
vectors_to_search = entities[-1][-2:]
search_params = {"metric_type": "L2", "params": {"nprobe": 10},}
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, output_fields=["random"])
result

In [26]:
# Performs a vector query ..
result = hello_milvus.query(expr="random > -14", output_fields=["random", "embeddings"])
result

[{'random': -13.0,
  'embeddings': [0.62187123,
   0.4412795,
   0.33215374,
   0.94789255,
   0.887061,
   0.32706463,
   0.41287848,
   0.17165068],
  'pk': 7},
 {'random': -13.0,
  'embeddings': [0.8586708,
   0.36688852,
   0.21213529,
   0.30698112,
   0.31122893,
   0.56188154,
   0.8043315,
   0.70717],
  'pk': 12},
 {'random': -12.0,
  'embeddings': [0.10162125,
   0.76658577,
   0.24427679,
   0.8723901,
   0.10867632,
   0.20757957,
   0.23539978,
   0.026004603],
  'pk': 13},
 {'random': -11.0,
  'embeddings': [0.72415304,
   0.16990551,
   0.19470866,
   0.88536435,
   0.292838,
   0.9057488,
   0.6311069,
   0.88532585],
  'pk': 14},
 {'random': -11.0,
  'embeddings': [0.2949666,
   0.39732966,
   0.359827,
   0.90880966,
   0.25295392,
   0.31431088,
   0.26185194,
   0.111722924],
  'pk': 17},
 {'random': -11.0,
  'embeddings': [0.33166495,
   0.18598574,
   0.4405827,
   0.62896,
   0.1035834,
   0.14615557,
   0.21164297,
   0.9709655],
  'pk': 19},
 {'random': -12.0,


In [27]:
# Performs a hybrid search ..
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, expr="random > -12", output_fields=["random"])
result

In [18]:
# Drops the collection ..
utility.drop_collection("hello_milvus")